In [2]:
from functions import scrape_target_site, init_db, DB_NAME, store_jobs, detect_job_changes
from parsers import timetohire_parser, hackernews_parser

In [3]:
site = "hackernews.com"
url = "https://news.ycombinator.com/newest"

soup = scrape_target_site(site, url)

In [4]:
soup

<html lang="en" op="newest"><head><meta content="origin" name="referrer"/><meta content="width=device-width, initial-scale=1.0" name="viewport"/><link href="news.css?kc7ZUPIx10J1yMolRDy2" rel="stylesheet" type="text/css"/>
<link href="y18.svg" rel="icon"/>
<meta content="noindex" name="robots"/>
<title>New Links | Hacker News</title></head><body><center><table bgcolor="#f6f6ef" border="0" cellpadding="0" cellspacing="0" id="hnmain" width="85%">
<tr><td bgcolor="#ff6600"><table border="0" cellpadding="0" cellspacing="0" style="padding:2px" width="100%"><tr><td style="width:18px;padding-right:4px"><a href="https://news.ycombinator.com"><img height="18" src="y18.svg" style="border:1px white solid; display:block" width="18"/></a></td>
<td style="line-height:12pt; height:10px;"><span class="pagetop"><b class="hnname"><a href="news">Hacker News</a></b>
<span class="topsel"><a href="newest">new</a></span> | <a href="front">past</a> | <a href="newcomments">comments</a> | <a href="ask">ask</a> 

In [5]:
hackernews_parser(soup, 'test.nl')

[{'id': '44299072',
  'title': 'NASA data reveals dramatic rise in intensity of weather events',
  'url': 'https://www.theguardian.com/world/2025/jun/17/nasa-data-reveals-dramatic-rise-in-intensity-of-weather-events',
  'timestamp': '2025-06-17T13:39:24 1750167564',
  'company': '',
  'source': 'Hacker News'},
 {'id': '44299070',
  'title': 'Planetary waves linked to wild summer weather have tripled since 1950',
  'url': 'https://apnews.com/article/extreme-weather-climate-change-heatwave-flooding-drought-cc4c3d93e178b6f4ab13cb7788998801',
  'timestamp': '2025-06-17T13:39:16 1750167556',
  'company': '',
  'source': 'Hacker News'},
 {'id': '44299060',
  'title': 'Noir is a language for private and verifiable computing',
  'url': 'https://noir-lang.org/',
  'timestamp': '2025-06-17T13:38:26 1750167506',
  'company': '',
  'source': 'Hacker News'},
 {'id': '44299055',
  'title': 'Sub-Second Postgres to Iceberg Mirroring',
  'url': 'https://github.com/Mooncake-Labs/moonlink',
  'timestamp'